<a href="https://colab.research.google.com/github/kumarbirendra/ai-ml-analysis/blob/master/INFY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import finance API and get historical stock data
!pip3 install yfinance
import yfinance as yf
import os
import json
import pandas as pd


infy = yf.Ticker("INFY.NS")
infy_hist = infy.history(period="max")

infy_hist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1996-01-01 00:00:00+05:30,0.537758,0.538948,0.535116,0.538948,204800,0.0,0.0
1996-01-02 00:00:00+05:30,0.537759,0.540402,0.536768,0.536768,204800,0.0,0.0
1996-01-03 00:00:00+05:30,0.540402,0.540402,0.540402,0.540402,102400,0.0,0.0
1996-01-04 00:00:00+05:30,0.535116,0.537759,0.535116,0.536834,307200,0.0,0.0
1996-01-05 00:00:00+05:30,0.530491,0.530491,0.530491,0.530491,51200,0.0,0.0
...,...,...,...,...,...,...,...
2022-10-17 00:00:00+05:30,1476.900024,1495.000000,1475.500000,1489.949951,4286396,0.0,0.0
2022-10-18 00:00:00+05:30,1510.000000,1510.949951,1497.250000,1503.699951,4692004,0.0,0.0
2022-10-19 00:00:00+05:30,1504.000000,1504.949951,1481.300049,1485.449951,3553171,0.0,0.0
